In [1]:
import requests
import pandas as pd
from flatten_json import flatten


def request_api(url, endpoint):
    response = requests.get(url + endpoint + '.json')

    return response.json()


def get_total_records(json_data):
    total_records = int(json_data['MRData']['total'])

    return total_records


def get_data_from_rounds(url, total):
    all_data = []

    for round_n in range(total+1):
        round_url = url + f'{round_n}/'
        data = request_api(round_url, 'results')
        circuit_info = flatten(data['MRData']['RaceTable']['Races'][0]['Circuit'])
        for results_data in data['MRData']['RaceTable']['Races'][0]['Results']:

            results_data.update(
                circuit_info
            )

            results_data = flatten(results_data)
            all_data.append(results_data)

    return all_data

def create_df(data):
    df = pd.DataFrame(data)

    return df


def normalize_columns_names(col):
    new_column_name = ""
    for val in col:
        if val.isupper() and new_column_name and new_column_name[-1] != "_":
            new_column_name += "_"
        new_column_name += val

    return new_column_name.lower()

In [2]:
SEASON = 2021

api_url = f"https://ergast.com/api/f1/{SEASON}/"

In [3]:
circuits_data = request_api(api_url, 'circuits')

In [4]:
number_of_rounds_on_season = get_total_records(circuits_data)

In [5]:
all_data = get_data_from_rounds(api_url, number_of_rounds_on_season)

In [6]:
df = create_df(all_data)

In [7]:
df.columns = [normalize_columns_names(col) for col in df.columns]

In [8]:
df

,number,position,position_text,points,driver_driver_id,driver_permanent_number,driver_code,driver_url,driver_given_name,driver_family_name,...,fastest_lap_time_time,fastest_lap_average_speed_units,fastest_lap_average_speed_speed,circuit_id,url,circuit_name,location_lat,location_long,location_locality,location_country
0,44,1,1,25,hamilton,44,HAM,http://en.wikipedia.org/wiki/Lewis_Hamilton,Lewis,Hamilton,...,1:34.015,kph,207.235,bahrain,http://en.wikipedia.org/wiki/Bahrain_Internati...,Bahrain International Circuit,26.0325,50.5106,Sakhir,Bahrain
1,33,2,2,18,max_verstappen,33,VER,http://en.wikipedia.org/wiki/Max_Verstappen,Max,Verstappen,...,1:33.228,kph,208.984,bahrain,http://en.wikipedia.org/wiki/Bahrain_Internati...,Bahrain International Circuit,26.0325,50.5106,Sakhir,Bahrain
2,77,3,3,16,bottas,77,BOT,http://en.wikipedia.org/wiki/Valtteri_Bottas,Valtteri,Bottas,...,1:32.090,kph,211.566,bahrain,http://en.wikipedia.org/wiki/Bahrain_Internati...,Bahrain International Circuit,26.0325,50.5106,Sakhir,Bahrain
3,4,4,4,12,norris,4,NOR,http://en.wikipedia.org/wiki/Lando_Norris,Lando,Norris,...,1:34.396,kph,206.398,bahrain,http://en.wikipedia.org/wiki/Bahrain_Internati...,Bahrain International Circuit,26.0325,50.5106,Sakhir,Bahrain
4,11,5,5,10,perez,11,PER,http://en.wikipedia.org/wiki/Sergio_P%C3%A9rez,Sergio,Pérez,...,1:33.970,kph,207.334,bahrain,http://en.wikipedia.org/wiki/Bahrain_Internati...,Bahrain International Circuit,26.0325,50.5106,Sakhir,Bahrain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,5,16,R,0,vettel,5,VET,http://en.wikipedia.org/wiki/Sebastian_Vettel,Sebastian,Vettel,...,1:34.030,kph,236.375,jeddah,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...,Jeddah Corniche Circuit,21.6319,39.1044,Jeddah,Saudi Arabia
436,11,17,R,0,perez,11,PER,http://en.wikipedia.org/wiki/Sergio_P%C3%A9rez,Sergio,Pérez,...,1:34.138,kph,236.104,jeddah,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...,Jeddah Corniche Circuit,21.6319,39.1044,Jeddah,Saudi Arabia
437,9,18,R,0,mazepin,9,MAZ,http://en.wikipedia.org/wiki/Nikita_Mazepin,Nikita,Mazepin,...,1:37.043,kph,229.036,jeddah,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...,Jeddah Corniche Circuit,21.6319,39.1044,Jeddah,Saudi Arabia
438,63,19,R,0,russell,63,RUS,http://en.wikipedia.org/wiki/George_Russell_(r...,George,Russell,...,1:36.130,kph,231.211,jeddah,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...,Jeddah Corniche Circuit,21.6319,39.1044,Jeddah,Saudi Arabia


In [9]:
len(df)

440